In [1]:
import numpy as np
import math

import pandas as pd   # csvファイルを読み込み、ndarrayに変換するために必要なる。

# 冷蔵庫　解説書準拠版

## 1. 定義

準寒冷地版ガイドラインクラス2の製品例2009年TOPの商品。
電動機消費電力87W、電熱装置150W、年間消費電力330kWh/年（JIS C9801-2006)

## 2. 動作モード

冷蔵庫の動作モードを以下に示す。

|動作モード|内容|説明|
|:---:|:---:|:---:|
|1|除霜|蒸発器表面に生じた着霜をヒーターで融解させる運転|
|2|休止||
|3|冷却|庫内に収納した内容物を冷却する|


## 3. 各動作モードでの消費電力

### 3.1. 日消費電力

冷蔵庫の日積算消費電力の実測値は、日平均雰囲気温度とJIS　C9801-1999　で規定される年間消費電力量から以下の式で計算される。

$$
E_{ E, ref, daily, d } = [(3.283 \times 10^{-3} -2.0 \times 10^{-6} \times E_{ E, ref, anuual\_ rtd\_1999, a})
\times (\theta_{ amb, ave, d }^{2} - 30 \times \theta_{ amb, ave, d }) + 1.85 \times 10^{-3} \times E_{ E, ref, anuual\_rtd\_1999, a} + 1.329 ] \times 10^{3}
$$  

$E_{ E, ref, daily, d }$ :日付$d$における冷蔵庫の日消費電力量 [kWh/d]  
$E_{ E, ref, annual\_trd\_1999, a}$ :JIS C9801-1999 で規定される冷蔵庫の年間費電力量 [kWh/yr]  
$\theta_{ amb, ave, d }$ :日付$d$の冷蔵庫の日平均雰囲気温度 [℃]  

In [2]:
def get_E_E_ref_daily_d( theta_o_ave_d, E_E_annual_rtd_2006_a):
    """冷蔵庫の日別消費電力量を取得する
    
    Parameters
    ----------
    theata_o_ave_d : ndarray(N-dimensional array)
        日平均外気温度[℃]を格納したND配列
        d日の日平均外気温度[℃]が年開始日から365個連続して格納されている

    E_E_annual_rtd_2006_a : float
        JIS C9801-2006 で規定される冷蔵庫の年間費電力量 [kWh/yr] 
        
    
    Returns
    ----------
    E_E_ref_daily__d : ndarray(N-dimensional array)
        冷蔵庫の日積算消費電力の実測値[Wh]を格納したND配列
        d日の冷蔵庫の日積算消費電力の実測値量[Wh]が年開始日から365個連続して格納されている
        
    """
    # 冷蔵庫の日平均雰囲気温度を取得する
    theta_amb_ave_d = get_theta_amb_ave_d( theta_o_ave_d )
    
    # JIS C9801-2006 で規定される冷蔵庫の年間費電力量をJIS C9801-1999 で規定される冷蔵庫の年間費電力量に変換する
    E_E_annual_rtd_1999_a = get_E_E_annual_rtd_1999_a( E_E_annual_rtd_2006_a )
    
    # 冷蔵の日別電力(kWh)を推定する
    E_E_ref_daily_d_kWh = (3.283 * 10**(-3) - 2.0 * 10**(-6) * E_E_annual_rtd_1999_a) \
                        * (theta_amb_ave_d**2 - 30 * theta_amb_ave_d) \
                        + 1.85 * 10**(-3) * E_E_annual_rtd_1999_a \
                        + 1.329
    
    # 冷蔵の日別電力を(Wh)推定する
    E_E_ref_daily_d = E_E_ref_daily_d_kWh * 10**(3)
    
    return E_E_ref_daily_d

#### 3.1.1. 冷蔵庫の日平均雰囲気温度

冷蔵庫の日平均雰囲気温度は、建築研究所実装実験棟101号室における日平均冷蔵庫雰囲気温度と日平均外気温度の実測値により求める。  
ただし、日平均外気温度は、地域、季節によらず14.6℃とする。

$$
\theta_{ amb, ave, d } = 0.4142 \times \theta_{ o, ave, d } + 15.47
$$  


$\theta_{ amb, ave, d }$ :日付$d$の冷蔵庫の日平均雰囲気温度 [℃] 
$\theta_{ o, ave, d }$ :日付$d$の日平外気温度（=14.6） [℃] 


In [3]:
def get_theta_amb_ave_d(theta_o_ave_d):
    """冷蔵庫の日平均雰囲気温度を取得する
    
    Parameters
    ----------
    theata_o_ave_d : ndarray(N-dimensional array)
        日別の平均外気温度[℃]を格納したND配列
        d日の平均外気温度[℃]が年開始日から365個連続して格納されている
    
    Returns
    ----------
    theata_amb_ave_d : ndarray(N-dimensional array)
        日別の冷蔵庫周辺空気の推定温度[℃]を格納したND配列
        d日の冷蔵庫周辺空気の推定温度[℃]が年開始日から365個連続して格納されている
    """
    
    # ただし、解説書では日別の平均外気温度は通年14.6℃に固定されているので以下の式で値を上書きする
    theta_o_ave_d = np.full(365, 14.6)
    
    # 冷蔵庫の日平均雰囲気温度を取得する
    theta_amb_ave_d = 0.4142 * theta_o_ave_d + 15.47
    
    return theta_amb_ave_d

#### 3.1.2. JIS C9801-2006に基づいた測定値の変換

JIS C9801-2006に基づき測定された値をJIS C9801-1999に基づき測定された値に変換するには、以下の式を利用する。

$$
E_{ E, ref, annual\_rtd\_1999, a } = \frac{ E_{ E, ref, annual\_rtd\_2006, a } }{ 3.48 }
$$  

$E_{ E, ref, annual\_trd\_1999, a}$ :JIS C9801-1999 で規定される冷蔵庫の年間費電力量 [kWh/yr]  
$E_{ E, ref, annual\_trd\_2006, a}$ :JIS C9801-2006 で規定される冷蔵庫の年間費電力量(=330) [kWh/yr]  
3.48 :JIS C9801-2006 で規定される冷蔵庫の年間費電力量をJIS C9801-1999 で規定される冷蔵庫の年間費電力量に換算するときの係数 [-]  


In [4]:
def get_E_E_annual_rtd_1999_a(E_E_annual_rtd_2006_a):
    """JIS C9801-2006 で規定される冷蔵庫の年間費電力量をJIS C9801-1999 で規定される冷蔵庫の年間費電力量に変換する
    
    Parameters
    ----------
    E_E_annual_rtd_2006_a : float
        JIS C9801-2006 で規定される冷蔵庫の年間費電力量 [kWh/yr] 
        
    Returns
    ----------
    E_E_annual_rtd_1999_a : float
        JIS C9801-1999 で規定される冷蔵庫の年間費電力量 [kWh/yr] 
    """

    # ただし、解説書ではJIS C9801-2006 で規定される冷蔵庫の年間費電力量は322 kW/yr に固定されているので以下の式で値を上書きする
    E_E_annual_rtd_2006_a = 330
    
    # JIS C9801-1999 で規定される冷蔵庫の年間費電力量に変換する
    E_E_annual_rtd_1999_a =  E_E_annual_rtd_2006_a / 3.48
    
    return E_E_annual_rtd_1999_a

### 3.2. 除霜運転時の15分間の消費電力量（動作モード1）

冷蔵庫の除霜運転時の15分間の消費電力量は以下の式より求める。

$$
E_{ E, ref_{ defrost }, d, t } = \frac{ P_{ E, ref_{ defrost }, d, t } \times 15}{ 60 }
$$  

$P_{ E, ref_{ defrost }, d, t }$ :冷蔵庫の除霜運転時の消費電力 [W]  
$E_{ E, ref_{ defrost }, d, t }$ :冷蔵庫の除霜運転時の15分間の消費電力量 [Wh]  


In [5]:
def get_E_E_ref_defrost_d_t( P_E_ref_defrost_rtd ):
    """冷蔵庫の除霜運転時の15分間の消費電力量を取得する
    
    Parameters
    ----------
    P_E_ref_defrost_rtd : float
        冷蔵庫の電熱装置の定格消費電力 [W] 
        
    Returns
    ----------
    E_E_ref_defrost_d_t : float
        冷蔵庫の除霜運転時の15分間の消費電力量 [Wh] 
    """
    
    # 冷蔵庫の除霜運転時の電熱装置の消費電力を求める
    P_E_ref_defrost_d_t = get_P_E_ref_defrost_d_t( P_E_ref_defrost_rtd )
    
    # 冷蔵庫の除霜運転時の15分間の消費電力量を求める
    E_E_ref_defrost_d_t =  ( P_E_ref_defrost_d_t * 15 ) / 60
    
    return E_E_ref_defrost_d_t

#### 3.2.1. 冷蔵庫の除霜運転時の電熱装置の消費電力

冷蔵庫の除霜運転時の消費電力は以下の式より求める。  
ただし、冷蔵庫の電熱装置の定格消費電力は150Wとする。

$$
P_{ E, ref_{ defrost }, d, t } = 0.9 \times P_{ E, ref_{ defrost }, rtd }
$$  

$P_{ E, ref_{ defrost }, rtd }$ :冷蔵庫の電熱装置の定格消費電力(=150) [W]  
$P_{ E, ref_{ defrost }, d, t }$ :冷蔵庫の除霜運転時の消費電力 [W]  


In [6]:
def get_P_E_ref_defrost_d_t( P_E_ref_defrost_rtd ):
    """冷蔵庫の除霜運転時の15分間の消費電力量を取得する
    
    Parameters
    ----------
    P_E_ref_defrost_rtd : float
        冷蔵庫の電熱装置の定格消費電力 [W] 
        
    Returns
    ----------
    P_E_ref_defrost_d_t : float
        冷蔵庫の除霜運転時の電熱装置の消費電力 [W] 
    """
    
    # 冷蔵庫の除霜運転時の電熱装置の消費電力を求める
    P_E_ref_defrost_d_t = 0.9 * P_E_ref_defrost_rtd
    
    # ただし、解説書では冷蔵庫の電熱装置の定格消費電力は150Wに固定されているので以下の式で値を上書きする
    P_E_ref_defrost_d_t = 0.9 * 150
    
    return P_E_ref_defrost_d_t

### 3.3. 除霜運転以外の日消費電力量

冷蔵庫の除霜運転以外の日消費電力量は以下の式より求める。

$$
E_{ E, ex\_defrost, daily, d } = E_{ E, ref, daily, d} - P_{ E, ref_{defrost}, d, t } \times t_{ ref_{defrost} }
$$  

$E_{ E, ex\_defrost, daily, d }$ :冷蔵庫の除霜運転以外の消費電力量 [Wh]  
$t_{ ref_{ defrost } }$ :冷蔵庫の除霜運転時間(=2) [h]  

In [7]:
def get_E_E_ex_defrost_daily_d( theta_o_ave_d, E_E_annual_rtd_2006_a, P_E_ref_defrost_rtd, t_ref_defrost ):
    """冷蔵庫の除霜運転以外の日消費電力量を取得する
    
    Parameters
    ----------
    theata_o_ave_d : ndarray(N-dimensional array)
        日別の平均外気温度[℃]を格納したND配列
        d日の平均外気温度[℃]が年開始日から365個連続して格納されている

    E_E_annual_rtd_2006_a : float
        JIS C9801-2006 で規定される冷蔵庫の年間費電力量 [kWh/yr] 

    P_E_ref_defrost_rtd : float
        冷蔵庫の電熱装置の定格消費電力 [W] 
        
    t_ref_defrost : float
        冷蔵庫の除霜運転時間 [h] 
    
    Returns
    ----------
    E_E_ex_defrost_daily_d : ndarray(N-dimensional array)
        日別の冷蔵庫の除霜運転以外の日消費電力量[Wh]を格納したND配列
        d日の冷蔵庫の除霜運転以外の日消費電力量[Wh]が年開始日から365個連続して格納されている
    """
    
    # 冷蔵庫の日別消費電力量を取得する
    E_E_ref_daily_d = get_E_E_ref_daily_d( theta_o_ave_d, E_E_annual_rtd_2006_a )
    
    # 冷蔵庫の除霜運転時の消費電力を取得する
    P_E_ref_defrost_d_t = get_P_E_ref_defrost_d_t( P_E_ref_defrost_rtd )
    
    # ただし、解説書では冷蔵庫の除霜運転時間は2時間（23～翌1）に固定されているので以下の式で値を上書きする
    t_ref_defrost = 2
    
    # 冷蔵庫の除霜運転時の電熱装置の消費電力を求める
    E_E_ex_defrost_daily_d = E_E_ref_daily_d - P_E_ref_defrost_d_t * t_ref_defrost
    
    return E_E_ex_defrost_daily_d

#### 3.3.1. 除霜運転を除いた運転時間

冷蔵庫の除霜運転を除いた運転時間は以下の式より求める。  
除霜運転時間は2時間で固定されているため、実質22時間の定数となる。

$$
t_{ ex\_defrost, daily } = 24 -  t_{ ref_{defrost} }
$$  


$t_{ ex\_defrost, daily }$ :冷蔵庫の除霜運転を除いた運転時間 [h]  

In [8]:
def get_t_ex_defrost_daily( t_ref_defrost ):
    """冷蔵庫の除霜運転を除いた運転時間を取得する
    
    Parameters
    ----------
    t_ref_defrost : float
        冷蔵庫の除霜運転時間 [h] 
    
    Returns
    ----------
    t_ex_defrost_daily : float
        冷蔵庫の除霜運転時間 [h] 
    """
        
    # ただし、解説書では冷蔵庫の除霜運転時間は2時間（23～翌1）に固定されているので以下の式で値を上書きする
    t_ref_defrost = 2
    
    # 冷蔵庫の除霜運転を除いた運転時間を取得する
    t_ex_defrost_daily = 24 - t_ref_defrost
    
    return t_ex_defrost_daily

### 3.4. 休止時の15分間の消費電力量（動作モード2）

冷蔵庫の休止時の消費電力は以下の式より求める。  

$$
E_{ E, ref_{ standby }, d, t } = \frac{ P_{ E, ref_{ standby }, d, t } \times 15 }{ 60 }
$$  


$E_{ E, ref_{ standby }, d, t }$ :冷蔵庫の休止時の15分間の消費電力量 [Wh]  
$P_{ E, ref_{ standby }, d, t }$ :冷蔵庫の休止時の消費電力(=10) [W]  

In [9]:
def get_E_E_ref_standby_d_t( P_E_ref_standby_d_t ):
    """冷蔵庫の休止時の15分間の消費電力を取得する
    
    Parameters
    ----------
    P_E_ref_standby_d_t : float
        冷蔵庫の休止時消費電力 [W] 
    
    Returns
    ----------
    E_E_ref_standby_d_t : float
        冷蔵庫の休止時の15分間の消費電力量 [Wh] 
    """
        
    # ただし、解説書では冷蔵庫の休止時消費電力は10Wに固定されているので以下の式で値を上書きする
    P_E_ref_standby_d_t = 10
    
    # 冷蔵庫の冷蔵庫の休止時の15分間の消費電力量を取得する
    E_E_ref_standby_d_t = ( P_E_ref_standby_d_t * 15) / 60
    
    return E_E_ref_standby_d_t

### 3.5. 冷却運転時の15分間の消費電力量（動作モード3）

冷蔵庫の休止時の消費電力は以下の式より求める。  

$$
t_{ ref_{ operation }, daily } = Int \left[ \frac{ t_{ ex\_defrost, daily} }{ \frac{ ( 15 + 15 \times n ) }{ 60 } } \right] \times \frac{ 15 \times n }{ 60 }
$$  

$$
t_{ ref_{ standby }, daily } = t_{ ex\_defrost, daily } - t_{ ref_{ operation }, daily }
$$  

$$
E_{ E, ref_{ operation }, d } = E_{ E, ex\_derost, daily, d } - t_{ ref_{ standby }, daily } \times P_{ E, ref_{ standby }, d, t }
$$  

$$
P_{ E, ref_{ operation }, d, t } = \frac{ E_{ E, ref_{ operation }, d } }{ t_{ ref_{ operation }, daily } }
$$  

$$
E_{ E, ref_{ operation }, d, t } = \frac{ P_{ E, ref_{ operation }, d, t } \times 15 }{ 60 }
$$  



$t_{ ref_{ operation }, daily }$ :日冷却運転時間 [h]  
n :運転1周期のうちの冷却運転の回数 [回]  
$t_{ ref_{ standby }, daily }$ :日休止時間 [h]  
$E_{ E, ref_{ operation }, d }$ :冷却運転時日電力量 [Wh]  
$E_{ E, ex\_defrost, daily, d }$ :除霜運転以外の日消費電力量 [Wh]  
$P_{ E, ref_{ operation }, d, t }$ :冷却運転時消費電力 [W]  
$E_{ E, ref_{ operation }, d, t }$ :冷却運転時の15分間の消費電力量 [Wh]




In [10]:
def get_E_E_ref_operation_d_t( P_E_ref_rtd, P_E_ref_standby_d_t, n, t_ref_defrost, theta_o_ave_d, E_E_annual_rtd_2006_a, P_E_ref_defrost_rtd ):
    """冷蔵庫の冷却時の15分間の消費電力量を取得する
    
    Parameters
    ----------
    P_E_ref_rtd : float
        冷蔵庫の電動機消費電力 [W] 
    
    P_E_ref_standby_d_t : float
        冷蔵庫の休止時の消費電力 [W] 
    
    n : float
        冷蔵庫の運転1周期のうちの冷却運転の回数 [回]
    
    t_ref_defrost : float
        冷蔵庫の日除霜運転時間 [h]
    
    theta_o_ave_d : ndarray(N-dimensional array)
        日別の平均外気温度[℃]を格納したND配列
        d日の平均外気温度[℃]が年開始日から365個連続して格納されている

    E_E_annual_rtd_2006_a : float
        JIS C9801-2006 で規定される冷蔵庫の年間費電力量 [kWh/yr] 

    P_E_ref_defrost_rtd : float
        冷蔵庫の電熱装置の定格消費電力 [W] 
    
    Returns
    ----------
    E_E_ref_operation_d_t : float
        冷蔵庫の冷却時の15分間の消費電力量 [Wh] 
    """
    
    # ただし、解説書では冷蔵庫の定格電動機消費電力は87Wに固定されているので以下の式で値を上書きする
    P_E_ref_rtd = 87
    
    # ただし、解説書では冷蔵庫の休止時消費電力は10Wに固定されているので以下の式で値を上書きする
    P_E_ref_standby_d_t = 10
    
    # ただし、解説書では運転1周期のうちの冷却運転の回数は2回に固定されているので以下の式で値を上書きする
    n = 2
    
    # ただし、解説書では冷蔵庫の除霜時間は2hに固定されているので以下の式で値を上書きする
    t_ref_defrost = 2
    
    # ただし、解説書では 日別の平均外気温度は14.6℃に固定されているので以下の式で値を上書きする
    theta_o_ave_d = np.full( 365, 14.6 )
    
    # ただし、解説書ではJIS C9801-2006 で規定される冷蔵庫の年間費電力量は330Wに固定されているので以下の式で値を上書きする
    E_E_annual_rtd_2006_a = 330
    
    # ただし、解説書では冷蔵庫の電熱装置の定格消費電力は150Wに固定されているので以下の式で値を上書きする
    P_E_ref_defrost_rtd = 150
    
    
    # 冷蔵庫の除霜運転を除いた運転時間を取得する
    t_ex_defrost_daily = get_t_ex_defrost_daily( t_ref_defrost )
    
    # 冷蔵庫の日冷却運転時間を取得する
    # 除霜運転を除いた運短時間を1周期に係る時間で割り、1日当たりの周期回数をINT関数で求め、それに1周期における冷却運転の時間を掛ける
    t_ref_operation_daily = int( t_ex_defrost_daily / ( ( 15 + 15 * n ) / 60 ) ) * ( 15 * n ) / 60
    
    # 冷蔵庫の日休止時間を取得する
    t_ref_standby_daily = t_ex_defrost_daily - t_ref_operation_daily

    # 冷蔵庫の除霜運転以外の日消費電力量を取得する
    E_E_ex_defrost_daily_d = get_E_E_ex_defrost_daily_d( theta_o_ave_d, E_E_annual_rtd_2006_a, P_E_ref_defrost_rtd, t_ref_defrost )
    
    # 冷蔵庫の冷却運転の日消費電力量を取得する
    E_E_ref_operation_d = E_E_ex_defrost_daily_d - P_E_ref_standby_d_t * t_ref_standby_daily
    
    # 冷蔵庫の冷却運転の消費電力を取得する
    P_E_ref_operation_d_t = E_E_ref_operation_d / t_ref_operation_daily
    
    # 冷蔵庫の冷却運転の消費電力が電動機消費電力を超えていないか確認する。
    # 現在は固定値なので異常値にはならないが、変動値とした場合には繰り返し計算が必要になるので、Webプログラム的には×では？
    #if P_E_ref_operation_d_t > P_E_ref_rtd:
    #    raise ValueError(P_E_ref_operation_d_t)
        
    
    # 冷蔵庫の冷蔵庫の冷却運転時の15分間の消費電力量を取得する
    E_E_ref_operation_d_t = ( P_E_ref_operation_d_t * 15 ) / 60
    
    return E_E_ref_operation_d_t

## ex. 1. 初期化

実装方法が固まってから検討する。現時点では無理。

## ex. 2.   一括処理

外部ファイルから参照する場合に実行する。

In [11]:
def get_parameters( ):
    """外部ファイルから参照する際に必要なパラメータを集める。
    
    Parameters
    ----------
    現時点（2019.11.06）では引数は内部で固定値に置換されるため、引数は無し。
    
    Returns
    ----------
    P_E_mode_1 : float
        冷蔵庫の除湿時の15分間の消費電力量 [Wh] 
    
    P_E_mode_2 : float
        冷蔵庫の休止時の15分間の消費電力量 [Wh] 
    
    P_E_mode_3 : ndarray(N-dimensional array)
        日別の冷蔵庫の冷却時の15分間の消費電力量[Wh]を格納したND配列
        d日の冷蔵庫の冷却時の15分間の消費電力量[Wh]が年開始日から365個連続して格納されている
    """
    
    # 除霜運転時の15分間の消費電力量（動作モード1） 現時点（2019.11.06）では引数は内部で固定値に置換されるため、何でもよい
    E_E_ref_defrost_d_t = get_E_E_ref_defrost_d_t(0)

    # 休止時の15分間の消費電力量（動作モード2）　現時点（2019.11.06）では引数は内部で固定値に置換されるため、何でもよい
    E_E_ref_standby_d_t = get_E_E_ref_standby_d_t(0)

    # 冷却時の15分間の消費電力量（動作モード3）　現時点（2019.11.06）では引数は内部で固定値に置換されるため、何でもよい
    E_E_ref_operation_d_t = get_E_E_ref_operation_d_t( 1, 2, 3, 4, np.zeros(365), 6, 7 )

    # 動作モード1の値を消費電力に変換する
    P_E_mode_1 = E_E_ref_defrost_d_t * 60 / 15
    
    # 動作モード2の値を消費電力に変換する
    P_E_mode_2 = E_E_ref_standby_d_t * 60 / 15
    
    # 動作モード3の値を消費電力に変換する
    P_E_mode_3 = E_E_ref_operation_d_t * 60 / 15
    
    return P_E_mode_1, P_E_mode_2, P_E_mode_3

## ex. 3.  外部ファイルからの参照関数

外部から家電スケジュールを受け取り、それに対応する秒数と消費電力を返す

In [12]:
def get_E_elc_HEA_steps(dictionary_of_HEA_schedule):
    """冷蔵庫の冷却時の15分間の消費電力量を取得する
    
    Parameters
    ----------
    dictionary_of_HEA_schedule : dictionary
        (key)計算ステップ番号 : ?float
        稼働開始時間 : datetime64
        稼働状態値(状態値の範囲は各家電に依存する) : ?float
        継続秒数 : ?float
    
    Returns
    ----------
    t_sec_steps : ndarray(N-dimensional array)
        ステップ別の継続秒数[s]を格納したND配列
        各ステップの継続秒数[s]が年開始日からステップ数分連続して格納されている
    
    P_elc_HEA_steps : ndarray(N-dimensional array)
        ステップ別に指定された動作モードでの家電の消費電力[W]を格納したND配列
        各ステップの動作モードでの家電の消費電力[w]が年開始日からステップ数分連続して格納されている
    """
    
    key_list = dictionary_of_HEA_schedule.keys()
    t_sec_steps = np.zeros(len(key_list))
    P_elc_HEA_steps = np.zeros(len(key_list))
    
    
    # 動作条件別の日別消費電力の読み出し(解説書の固定値を採用しなくなった場合には、引数を再調整する事)
    P_elc_mode_1, P_elc_mode_2, P_elc_mode_3 = get_parameters()
    
    for i in range(len(key_list)):
        t_sec_steps[i] = dictionary_of_HEA_schedule[i][2]
        
        # 受け取った開始日時から年開始日からの日数を取得する
        t = pd.to_datetime( dictionary_of_HEA_schedule[i][0] ) - pd.to_datetime( np.datetime64('2002-01-01T00:00:00') )
        
        if dictionary_of_HEA_schedule[i][1] == 1:
            P_elc_HEA_steps[i] = P_elc_mode_1
        elif dictionary_of_HEA_schedule[i][1] == 2:
            P_elc_HEA_steps[i] = P_elc_mode_2
        elif dictionary_of_HEA_schedule[i][1] == 3:
            P_elc_HEA_steps[i] = P_elc_mode_3[t.days]
        else:
            raise ValueError(item)
    
    return t_sec_steps, P_elc_HEA_steps

## 外部ファイルからの参照テスト

In [13]:
stp = np.array([0,1,2])
tm = np.array(['2001-01-01T00:00:00', '2001-01-01T00:15:00', '2001-12-31T23:45:00'], dtype='datetime64')
stat = np.array([3,1,2])
sec = np.array([(np.datetime64(tm[1]) - np.datetime64(tm[0])).item().total_seconds(), \
                (np.datetime64(tm[2]) - np.datetime64(tm[1])).item().total_seconds(), \
                (np.datetime64('2002-01-01T00:00:00') - np.datetime64(tm[2])).item().total_seconds()])
schedule_of_HEA = dict(zip(stp, zip(tm, stat, sec)))

print(schedule_of_HEA)

{0: (numpy.datetime64('2001-01-01T00:00:00'), 3, 900.0), 1: (numpy.datetime64('2001-01-01T00:15:00'), 1, 31534200.0), 2: (numpy.datetime64('2001-12-31T23:45:00'), 2, 900.0)}


In [14]:
get_E_elc_HEA_steps(schedule_of_HEA )

(array([  9.00000000e+02,   3.15342000e+07,   9.00000000e+02]),
 array([  41.02204777,  135.        ,   10.        ]))